<a href="https://colab.research.google.com/github/Anuj-AD/SIPPP/blob/main/Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Try out different investing strategies on historical stock market data.

Phase 1:
Comparing monthly SIP in index fund vs daily investing strategy in ETF. Let's call this strategy SIP++.

> What is SIP++: let's say i plan to invest INR 365000 per year in 12 monthly installments. (INR 1000 per day). SIP++ says that instead of investing monthly, invest daily. However the rule to follow is to invest only if stock market went down in the last day. eg: let's say stock market went up for 3 days, and went down on 4th day. As per SIP++, Invest 4000 on day 4 directly.

In [ ]:
# prompt: import the daily price data of NIFTY50 for the last "N" years in a table.
#import data of the stock in a dataframe called stick_data

!pip install yfinance
import yfinance as yf
import numpy as np
import pandas as pd

years = 1 # years
ticker = "SETFNIF50.NS"
stock_data = yf.download(ticker, period = f"{years}y", progress=False)

In [ ]:
# define a column called percentage change to calculate the change from previous day.
stock_data['percentage_change'] = ((stock_data['Close'] - stock_data['Open'])/stock_data['Open'])*100

In [ ]:
# find the first day of each month in this data, find the highest value on the day, assume i bought the stock at highest value, find the total number of stocks i can buy.
# Floor it down to nearest integer, save this in stock_quantity. add this to get total_stock, multiply this by closing price of last day to get total value.

first_days = stock_data.resample('MS').first()
stock_quantity = np.floor((366000/12)/first_days['High'])
# 1000 per day equals 365000 per year. I am taking 366000 for leap year, and keeping this number same for non-leap years. basically, giving advantage to SIPs.
total_stock = sum(stock_quantity)
total_money = total_stock*stock_data.iloc[-1]['Close']

#we have successfully made a SIP calculator. now the difficult part

In [ ]:
sippp_total_stock = 0 #this will contain the total amount of stock i was able to buy
sippp_total_money = 0 #we will calculate the total here to compare with standard SIP total.

In [ ]:
# prompt: add a column in stock data names sippp_invest, set it to 0 if percentage_change >=0, set it to 1 if percentage change is <0
stock_data['sippp_invest'] = np.where(stock_data['percentage_change'] >= 0, 0, 1)
stock_data['sippp_invest_new'] = np.where(stock_data['sippp_invest'].shift(1) == 1, 1, 0)

stock_data['counter'] = 0

In [ ]:
i = 0
for i in range(len(stock_data)):
  if i >= len(stock_data):
    break
  elif i == 0:
    stock_data.iloc[i, stock_data.columns.get_loc('counter')] = 1
  elif stock_data.iloc[i, stock_data.columns.get_loc('sippp_invest_new')] == 0:
    stock_data.iloc[i, stock_data.columns.get_loc('counter')] = stock_data.iloc[i-1, stock_data.columns.get_loc('counter')] + 1
  elif stock_data.iloc[i, stock_data.columns.get_loc('sippp_invest_new')] == 1:
    stock_data.iloc[i, stock_data.columns.get_loc('counter')] = 0

stock_data['invest'] = np.where(stock_data['sippp_invest_new'] == 1, stock_data['counter'].shift(1) + 1, 0)


In [ ]:

stock_data['money'] = (366000 * years)/ len(stock_data)

stock_data['sippp_quantity'] = np.floor(stock_data['sippp_invest_new'] * stock_data['invest'] * stock_data['money'] / stock_data['Open'])
stock_data['sippp_invested'] = stock_data['sippp_quantity'] * stock_data['Open']
sippp_total_stock = sum(stock_data['sippp_quantity'])
sippp_total_money = sippp_total_stock * stock_data.iloc[-1]['Close']
print("SIP: ", total_money)
print("SIPPP: ", sippp_total_money)
invested_amount = 366000 * years
sippp_invested_amount = sum(stock_data['sippp_invested'])
return_sip = (total_money-invested_amount)*100/invested_amount

return_sippp = (sippp_total_money-sippp_invested_amount)*100/sippp_invested_amount

print("SIP Returns: ", return_sip)
print("SIPPP Returns: ", return_sippp)

print(invested_amount, sippp_invested_amount)

SIP:  429765.4963989258
SIPPP:  383016.63787841797
SIP Returns:  17.42226677566278
SIPPP Returns:  8.900927381406762
366000 351711.08923339844
